<a href="https://colab.research.google.com/github/rendalamili/ml-for-table-extraction/blob/main/Table_Transformer_Model_Training_Debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipdb
import ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.4 MB/s eta 0:00:00


In [ ]:
%pdb on

Automatic pdb calling has been turned ON


In [ ]:
# Install packages
!pip install camelot-py[cv] ghostscript PyPDF2==1.26.0 pdf2image transformers matplotlib numpy opencv-python pandas torch tensorflow table-transformer transformers datasets
!apt-get install -y ghostscript
!pip install --upgrade scipy
!pip install --upgrade camelot-py[cv]
!pip install pytesseract
!apt-get install -y tesseract-ocr
!apt-get install -y poppler-utils
!pip uninstall pillow -y
!pip install pillow
!pip install timm
#!kaggle datasets download -d sreesankar711/pubtables-subset-100k
#!unzip pubtables-subset-100k.zip -d /content/pubtables
!kaggle datasets download -d davidtschirschwitz/construction-industry-steel-ordering-lists-cisol
!unzip construction-industry-steel-ordering-lists-cisol.zip -d /content/steel_ordering_lists



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of camelot-py[cv] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of table-transformer to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.4/582.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common libidn12 libijs-0.35
  libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 49 not upgraded.
Need to get 16.7 MB of archives.
After this operation, 63.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-droid-fallback all 1:6.0.1r16-1.1build1 [1,805 kB]
Ge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.7 MB/s eta 0:00:00
Dataset URL: https://www.kaggle.com/datasets/davidtschirschwitz/construction-industry-steel-ordering-lists-cisol
License(s): Attribution 4.0 International (CC BY 4.0)
100% 735M/736M [00:43<00:00, 21.7MB/s]
100% 736M/736M [00:43<00:00, 17.7MB/s]
Archive:  construction-industry-steel-ordering-lists-cisol.zip
  inflating: /content/steel_ordering_lists/cisol_TD-TSR/TD-TSR/annotations/desktop.ini  
  inflating: /content/steel_ordering_lists/cisol_TD-TSR/TD-TSR/annotations/train.json  
  inflating: /content/steel_ordering_lists/cisol_TD-TSR/TD-TSR/annotations/val.json  
  inflating: /content/steel_ordering_lists/cisol_TD-TSR/TD-TSR/desktop.ini  
  inflating: /content/steel_ordering_lists/cisol_TD-TSR/TD-TSR/images/desktop.ini  
  inflating: /content/steel_ordering_lists/cisol_TD-TSR/TD-TSR/images/test/09_5903EUe093_183_1991a_Bew_GWW_Sued

In [ ]:
# Import libraries
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PDF libaries
import camelot
import ghostscript
import PyPDF2 as pypdf
from pdf2image import convert_from_path

# Image processsing libaries
import pytesseract
import cv2
import PIL
from PIL import Image
from torchvision.datasets import ImageFolder
from torchvision import transforms
from PyPDF2 import PdfFileReader

# Pytorch libraries
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Transformer and datasets
import transformers
from transformers import TableTransformerForObjectDetection, Trainer, TrainingArguments, pipeline
from transformers import AutoImageProcessor, TableTransformerForObjectDetection, TrainingArguments, Trainer
from transformers import DefaultDataCollator
from transformers import Trainer, TrainingArguments, default_data_collator
import datasets
from datasets import load_dataset
from datasets import Dataset, Features, ClassLabel, Value, Image as ImageType
from PIL import Image
import torch
from torchvision import transforms
from transformers import TableTransformerForObjectDetection

# Google Collab
from google.colab import files

In [ ]:
# Load pre-trained Table Transformer model
model = TableTransformerForObjectDetection.from_pretrained('microsoft/table-transformer-detection')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at microsoft/table-transformer-detection were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint

In [ ]:
# Load image processor
image_processor = AutoImageProcessor.from_pretrained("microsoft/table-transformer-detection")

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [ ]:
# Load training dataset
train_dataset = load_dataset('imagefolder', data_dir='/content/steel_ordering_lists/cisol_TD-TSR')

Resolving data files:   0%|          | 0/507 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/169 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/171 [00:00<?, ?it/s]

In [ ]:
# Test dataset
print(train_dataset['train'][0])

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2480x3509 at 0x7ECE561BF7F0>, 'label': None}


In [ ]:
# Subset of 100 examples from the training dataset
subset_training_dataset = train_dataset['train'].select(range(100))

In [ ]:
# Test dataset
for sample in subset_training_dataset:
    print(sample)

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2480x3509 at 0x7ECD1C3C0070>, 'label': None}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2480x3509 at 0x7ECD1C313EE0>, 'label': None}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2480x3509 at 0x7ECD1C313EB0>, 'label': None}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2480x3509 at 0x7ECD1C313E80>, 'label': None}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2480x3509 at 0x7ECD1C313E50>, 'label': None}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2480x3509 at 0x7ECD1C313D90>, 'label': None}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2480x3509 at 0x7ECD1E58BB50>, 'label': None}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2480x3509 at 0x7ECD1E58BA30>, 'label': None}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2480x3509 at 0x7ECD1E58B910>, 'label': None}
{'image': <PIL.PngImagePlugi

In [ ]:
def preprocess_function(examples):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)), # Resize image
        transforms.ToTensor(),         # Convert image to Pytorch tensor
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalise pixel value
    ])

    # Find colum in dataset that contains image paths or image objects
    image_column = 'image_path'
    if image_column not in examples:
        image_column = 'image'  # Fallback to 'image' column
        if image_column not in examples:
            raise KeyError(f"Neither 'image_path' nor 'image' found in the dataset columns: {list(examples.keys())}")

    if 'pixel_values' in examples:
        return examples

    # Initialise 'pixel_values' as tensor list
    examples['pixel_values'] = []

    # Check image objects
    for image in examples[image_column]:
        try:
            # If PIL Image, convert to RGB
            pil_image = image.convert("RGB")
            # Apply transformations
            examples['pixel_values'].append(transform(pil_image))
        except (TypeError, ValueError, IOError, AttributeError) as e:
            print(f"Error processing image: {e}, type: {type(image)}")
            examples['pixel_values'].append(torch.zeros(3, 224, 224))  # Placeholder

    # Convert list of tensors to single tensor
    examples['pixel_values'] = torch.stack(examples['pixel_values'])

    return examples

In [ ]:
# Apply preprocessing function to the subset dataset
subset_training_dataset = subset_training_dataset.map(preprocess_function, batched=True)

In [ ]:
# Test preprocessing
print(subset_training_dataset[0].keys())

dict_keys(['image', 'label', 'pixel_values'])


In [ ]:
# Prepare batch for training
def custom_data_collator(features):
    # Initialise dictionary to hold batch data
    batch = {}

    # Check for pixel_values key and handle accordingly
    # Stack pixel value into tensor
    if 'pixel_values' in features[0].keys():
        batch['pixel_values'] = torch.stack([torch.tensor(f['pixel_values']) for f in features])
    else:
        print("Warning: 'pixel_values' key not found in dataset.")
        return None

    # Handle labels
    if 'labels' in features[0]:
        batch['labels'] = [f['labels'] for f in features]

    return batch

In [ ]:
# Training arguments
training_arguments = TrainingArguments(
    output_dir='./results',                    # Model prediction directory
    eval_strategy='epoch',                     # Eval strategy during training
    save_strategy='epoch',                     # Save strategy during training
    save_total_limit=2,                        # Maximum number of checkpoints to save
    save_steps=100,                            # Save every 100 steps
    learning_rate=2e-5,                        # Learning rate
    per_device_train_batch_size=16,            # Training batch size
    per_device_eval_batch_size=16,             # Evaluation batch size
    num_train_epochs=3,                        # Training epochs
    logging_dir='./logs',                      # Logging directory
    logging_steps=10,                          # Log training
    remove_unused_columns=False,               # Do not remove unused columns in dataset
)

In [ ]:
# Initialise Trainer for model training
trainer = Trainer(
    model=model,                               # Trained model
    args=training_arguments,                   # Training arguments
    train_dataset=subset_training_dataset,     # Training dataset
    data_collator=custom_data_collator         # Data collator for batch
    #eval_dataset=val_dataset['train'],        # Training validation data set
)


In [ ]:
# Train model
trainer.train()

ValueError: The model did not return a loss from the inputs, only the following keys: logits,pred_boxes,last_hidden_state,encoder_last_hidden_state. For reference, the inputs it received are pixel_values.

> /usr/local/lib/python3.10/dist-packages/transformers/trainer.py(3381)compute_loss()
   3379         else:
   3380             if isinstance(outputs, dict) and "loss" not in outputs:
-> 3381                 raise ValueError(
   3382                     "The model did not return a loss from the inputs, only the following keys: "
   3383                     f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."



In [ ]:
# Test image on model
table_recogniser = pipeline('table-recognition', model=model)

# Load table image
test_image = Image.open('table_0.png')
table_recogniser(test_image)

# Convert prediction to JSON
predicted_tables = json.dumps(table_recogniser(test_image))
with open('predicted_tables.json', 'w') as f:
  f.write(predicted_tables)

print(prediction_json)